### Prediction File

#### Importing Libraries 

In [40]:
import cv2 as cv
import numpy as np
import os
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from scipy.cluster.vq import *
from imutils import paths
%matplotlib inline

#### Load Classifier For Prediction

In [41]:
DATA_FILENAME = "Fruit_CLassifier.pkl"
clf, classes_names, stdSlr, k, voc, idf = joblib.load(DATA_FILENAME)
print("Data loaded! ("+DATA_FILENAME+")")

Data loaded! (Fruit_CLassifier.pkl)


#### Read Test Data

In [42]:
test_path = '..\data\Test_data'
image_paths = []
testing_categories = os.listdir(test_path)
for test_category in testing_categories:
    dir = os.path.join(test_path, test_category)
    class_path = list(paths.list_images(dir))
    image_paths+=class_path

#### Calculate KeyPoint and Descriptor of Testing Data

In [43]:
sift = cv.xfeatures2d.SIFT_create()
des_list = []
for image_path in image_paths:
    im = cv.imread(image_path)
    if im.all == None:
        print("No such file {}\nCheck if the file exists".format(image_path))
    kpts, des = sift.detectAndCompute(im, None)
    des_list.append((image_path, kpts, des))

#### Stack All Desciptors/Keypoints Vertically in Numpy Array

In [44]:
descriptors = des_list[0][2]
for image_path, keypoint, descriptor in des_list[0:]:
    descriptors = np.vstack((descriptors, descriptor))

#### Calculate Histogram for Test Images Data

In [45]:
test_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    words, distance = vq(des_list[i][2],voc)
    for w in words:
        test_features[i][w] += 1
nbr_occurences = np.sum(test_features > 0, axis = 0)
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

#### Normalization of Features 

In [46]:
test_features = test_features * idf
test_features = stdSlr.transform(test_features)

#### Let's Start Prediction 

In [47]:
predictions =  [classes_names[i] for i in clf.predict(test_features)]
print(predictions)

['Apple', 'Apple', 'Mango', 'Mango']


#### Main Prediction Cell It May take some time depend on Processing Power

After Running Below cell image screen open you need to enter s from your keyboard to store predicted image and move to next image

In [48]:
saved_imgs_counter = 0
for image_path, prediction in zip(image_paths, predictions):
    image = cv.imread(image_path)
    cv.namedWindow("Image", cv.WINDOW_NORMAL)
    pt = (0, 3 * image.shape[0] // 4)
    cv.putText(image, prediction, pt ,cv.FONT_ITALIC, 1, [0, 0, 255], 2)
    cv.imshow("Image", image)
    k = cv.waitKey(0)
    if(k == 27):
        cv.destroyAllWindows()
    elif(k == ord('s')):
        cv.imwrite("./Predicted_images/predict_"+str(saved_imgs_counter)+".jpg", image)
        saved_imgs_counter = saved_imgs_counter + 1
        cv.destroyAllWindows()

#### Save All output in Montage Form

After Running Below cell image screen open you need to enter s from your keyboard to store predicted image

In [49]:
from imutils import build_montages
import random
imagePaths = [i for i in paths.list_images('./output/')]
random.shuffle(imagePaths)
images = []
for imagePath in imagePaths:
    image = cv.imread(imagePath)
    images.append(image)
montages = build_montages(images, (256, 256), (3, 3))
for montage in montages:
    cv.imshow("Montage", montage)
    k = cv.waitKey(0)
    if(k == 27):
        cv.destroyAllWindows()
    elif(k == ord('s')):
        cv.imwrite("./Predicted_images/predict_"+str(saved_imgs_counter)+".jpg", image)
        saved_imgs_counter = saved_imgs_counter + 1
        cv.destroyAllWindows()